In [1]:
# !pip install tensorly

In [2]:
import numpy as np
from ncon import ncon

import numpy as np
np.set_printoptions(formatter={
    'float': lambda x: "{0:+0.5f}".format(x), 
    'double': lambda x: "{0:0.5d}".format(x),
    'complexfloat': lambda x: "{0.real:0.5f}".format(x) + '+ '[int(x.imag < 0)] + "{0.imag:0.5f}j".format(x)
})

In [3]:


PLUS = np.array([1, 1]) * .5 ** .5
PP = np.kron(PLUS, PLUS)
PPT = PP.reshape(2, 2)

BELL = np.array([0.6, 0, 0, -0.4])
BELLPHI = np.array([0, 1, 1, 0]) * .5 ** .5
BELLT = BELL.reshape(2, 2)
BELLPHIT = BELLPHI.reshape(2, 2)

BB = np.kron(BELL, BELL).reshape(2, 2, 2, 2)
PPPPT = np.kron(PP, PP).reshape(2, 2, 2, 2)

FENT = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]).reshape(2, 2, 2, 2)
FENT = FENT / np.linalg.norm(FENT.reshape(-1))

E = np.array([[0, 1], [-1, 0]])

GHZP = np.kron(np.array([1, 0, 0, 0, 0, 0, 0, 1]), PLUS).reshape(2, 2, 2, 2) * .5 ** .5

PARTIALLY = np.kron(np.array([0.6, 0.4, 0.4, 0.6]), np.array([0.6, 0.4, 0.4, -0.6])).reshape(2, 2, 2, 2)
PARTIALLY = PARTIALLY / np.linalg.norm(PARTIALLY.reshape(-1))

W = np.array([0, 1, 1, 0, 1, 0, 0, 0]) * ((1/3) ** .5)
WP = np.kron(W, PLUS).reshape(2, 2, 2, 2)

ZERO = np.array([1, 0])
ONE = np.array([0, 1])

In [4]:
COPY = np.zeros((2, 2, 2))
COPY[0, 0, 0] = 1
COPY[1, 1, 1] = 1
print(ncon(COPY, [1, 1, -1]))
print(ncon([COPY, E], [[1, 2, -1], [1, 2]]))

[+1.00000 +1.00000]
[+0.00000 +0.00000]


In [5]:
# contracting BELL state with itself: measured in it's basis
ncon((BELLT, BELLT), [[1, 2], [1, 2]])

array(+0.52000)

In [6]:
# contracting BELL state with |++>: measured in it's basis
ncon((BELLT, PPT), [[1, 2], [1, 2]])

array(+0.10000)

In [7]:
# contracting BELL state with |PHI+>: measured in it's basis
ncon((BELLT, BELLPHIT), [[1, 2], [1, 2]])

array(+0.00000)

In [8]:
# compute matrix determinant
ncon((BELLT.T, E, BELLT), [[-1, 1], [1, 2], [2, -2]])

array([[+0.00000, -0.24000],
       [+0.24000, +0.00000]])

In [9]:
# compute concurrence of a bell state. Concurrence is a module
abs(ncon((BELLT, E, E, BELLT.conj()), [[1, 2], [1, 3], [2, 4], [3, 4]]))

0.48

In [10]:
# compute concurrence of |++> state
abs(ncon((PPT, E, E, PPT.conj()), [[1, 2], [1, 3], [2, 4], [3, 4]]))

0.0

In [11]:
# Contract double-bell state to measure its concurrence
# but this is irrelevant, as this for 2-state only
abs(ncon((BB, E, E, E, E, BB.conj()), [
    [1, 2, 3, 4], # BB
    [1, 5],  # eps0
    [2, 6], # eps1
    [3, 7], # eps2
    [4, 8], # eps3
    [5, 6, 7, 8]# BB
]))

0.23040000000000002

In [12]:
# pairwise

abs(ncon((BB, E, E, BB.conj()), [
    [1, 2, 3, 4], # BB
    [1, 5],  # eps0
    [2, 6], # eps1
    [5, 6, 3, 4]# BB.conj() wired
]))

0.2496

In [13]:
# pairwise

abs(ncon((BB, E, E, BB.conj()), [
    [1, 2, 3, 4], # BB
    [1, 5],  # eps0
    [3, 6], # eps1
    [5, 2, 6, 4]# BB.conj() wired
]))

0.0

Using $\epsilon$ tensor for more than 2 modes is illegal. There is a more sphisticated approach. But let us see if it shows anything.

In [14]:
# Contract partially-entangled state to measure its concurrence
ncon((PARTIALLY, E, E, E, E, PARTIALLY.conj()), [
    [1, 2, 3, 4], # BB
    [1, 5],  # eps0
    [2, 6], # eps1
    [3, 7], # eps2
    [4, 8], # eps3
    [5, 6, 7, 8]# BB
])

array(-0.38462)

In [15]:
# Contract FULLY-ent state to measure its concurrence
ncon((FENT, E, E, E, E, FENT.conj()), [
    [1, 2, 3, 4], # BB
    [1, 5],  # eps0
    [2, 6], # eps1
    [3, 7], # eps2
    [4, 8], # eps3
    [5, 6, 7, 8]# BB
])

array(+1.00000)

In [16]:
# Contract WP state to measure its concurrence
ncon((WP, E, E, E, E, WP.conj()), [
    [1, 2, 3, 4], # BB
    [1, 5],  # eps0
    [2, 6], # eps1
    [3, 7], # eps2
    [4, 8], # eps3
    [5, 6, 7, 8]# BB
])

array(+0.00000)

In [17]:
# contract core tensor of BELL's core to |0> and |1>
import tensorly as tl
from tensorly import decomposition
core, matrices = tl.decomposition.tucker(BB, rank=BB.shape)

ncon([core, ZERO, ZERO, ZERO, ZERO], [[1, 2, 3, 4], [1], [2], [3], [4]]), \
ncon([core, ONE, ONE, ONE, ONE], [[1, 2, 3, 4], [1], [2], [3], [4]])

(array(+0.36000), array(+0.16000))

In [18]:
# contract core tensor of +'s core to |0> and |1>
core, matrices = tl.decomposition.tucker(PPPPT, rank=PPPPT.shape)

ncon([core, ZERO, ZERO, ZERO, ZERO], [[1, 2, 3, 4], [1], [2], [3], [4]]), \
ncon([core, ONE, ONE, ONE, ONE], [[1, 2, 3, 4], [1], [2], [3], [4]])

(array(+1.00000), array(+0.00000))

In [19]:
# contract core tensor of 0000+1111 to |0> and |1>

core, matrices = tl.decomposition.tucker(FENT, rank=FENT.shape)
ncon([core, ZERO, ZERO, ZERO, ZERO], [[1, 2, 3, 4], [1], [2], [3], [4]]), \
ncon([core, ONE, ONE, ONE, ONE], [[1, 2, 3, 4], [1], [2], [3], [4]])

(array(+0.70711), array(+0.70711))

In [20]:
# contract core tensor of GHZ+P to |0> and |1>

core, matrices = tl.decomposition.tucker(GHZP, rank=GHZP.shape)
ncon([core, ZERO, ZERO, ZERO, ZERO], [[1, 2, 3, 4], [1], [2], [3], [4]]), \
ncon([core, ONE, ONE, ONE, ONE], [[1, 2, 3, 4], [1], [2], [3], [4]])

(array(-0.70711), array(+0.00000))

In [21]:
# contract core tensor of W+P to |0> and |1>

core, matrices = tl.decomposition.tucker(WP, rank=WP.shape)
ncon([core, ZERO, ZERO, ZERO, ZERO], [[1, 2, 3, 4], [1], [2], [3], [4]]), \
ncon([core, ONE, ONE, ONE, ONE], [[1, 2, 3, 4], [1], [2], [3], [4]])

(array(+0.00000), array(+0.00000))

# Proposed methods

In [22]:
def free_leg_test(T):
    # array legs contracted
    result = np.zeros((T.ndim,))

    plus = np.array([1, 1]) * .5 ** .5
    one = np.array([0, 1])

    c, m = decomposition.tucker(T, rank=T.shape)
    cabs = abs(c)
    mc = m.copy()
    
    for i in range(T.ndim):
        
        # for each mode i we will replace m[i] with I tensor
        # while for the others we will contract to |+>
        measurements = [plus] * T.ndim
        measurements[i] = one
        
        tensors = [cabs] + measurements
        
        contractions = []
        # outgoing contractions of core tensor
        contractions.append(list(range(1, T.ndim + 1)))
        # measurements
        contractions += [[q] for q in contractions[0]]
        # hang a free leg
        result[i] = ncon(tensors, contractions)
    return result

In [23]:
def test_tensor(T):
    result = np.zeros((T.ndim, T.ndim))

    plus = np.array([1, 1]) * .5 ** .5
    eps = np.array([0, 1, -1, 0]).reshape(2, 2)
    
    for i in range(T.ndim):
        for j in range(i + 1, T.ndim):        
            tensors = [T, eps, eps, T.conj()]

            contractions = []
            # outgoing contractions of core tensor
            contractions.append(list(range(1, T.ndim + 1)))
            # 2 wires to eps
            contractions.append([i + 1, T.ndim + 1])
            contractions.append([j + 1, T.ndim + 2])
            # incoming
            incoming = list(range(1, T.ndim + 1))
            incoming[i] = T.ndim + 1
            incoming[j] = T.ndim + 2
            contractions.append(incoming)
            # print(contractions)
            result[i, j] = abs(ncon(tensors, contractions))
            result[j, i] = result[i, j]
    return result

In [24]:
print("GHZ+P\n", free_leg_test(GHZP))
print("W+P\n", free_leg_test(WP))
print("BELL\n", free_leg_test(BELLT))
print("BELL+BELL\n", free_leg_test(BB))
print("++++\n", free_leg_test(PPPPT))

GHZ+P
 [+0.25000 +0.25000 +0.25000 +0.00000]
W+P
 [+0.20412 +0.20412 +0.20412 +0.00000]
BELL
 [+0.28284 +0.28284]
BELL+BELL
 [+0.14142 +0.14142 +0.14142 +0.14142]
++++
 [+0.00000 +0.00000 +0.00000 +0.00000]


In [25]:
print("GHZ+P\n", test_tensor(GHZP))
print("W+P\n", test_tensor(WP))
print("BELL\n", test_tensor(BELLT))
print("BELL+BELL\n", test_tensor(BB))
print("++++\n", test_tensor(PPPPT))

GHZ+P
 [[+0.00000 +0.00000 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.00000]]
W+P
 [[+0.00000 +0.66667 +0.66667 +0.00000]
 [+0.66667 +0.00000 +0.66667 +0.00000]
 [+0.66667 +0.66667 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.00000]]
BELL
 [[+0.00000 +0.48000]
 [+0.48000 +0.00000]]
BELL+BELL
 [[+0.00000 +0.24960 +0.00000 +0.00000]
 [+0.24960 +0.00000 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.24960]
 [+0.00000 +0.00000 +0.24960 +0.00000]]
++++
 [[+0.00000 +0.00000 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.00000]
 [+0.00000 +0.00000 +0.00000 +0.00000]]
